In [1]:
import os

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from torch.utils.data import DataLoader

# from shared.collator import zero_pad_collator
# from shared.tokenizers import HamNoSysTokenizer
# from data import get_dataset
# from model import IterativeTextGuidedPoseGenerationModel

c:\users\nguyen thanh loc\appdata\local\programs\python\python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Collator

In [2]:
from typing import Dict, List, Tuple, Union, TypedDict

import numpy as np
import torch
from pose_format.torch.masked import MaskedTensor, MaskedTorch


In [3]:
def collate_tensors(batch: List, pad_value=0):
    datum = batch[0]

    if isinstance(datum, dict):  # Recurse over dictionaries
        return zero_pad_collator(batch)

    if isinstance(datum, (int, np.int32)):
        return torch.tensor(batch, dtype=torch.long)

    if isinstance(datum, (MaskedTensor, torch.Tensor)):
        max_len = max(len(t) for t in batch)
        if max_len == 1:
            return torch.stack(batch)

        torch_cls = MaskedTorch if isinstance(datum, MaskedTensor) else torch

        new_batch = []
        for tensor in batch:
            missing = list(tensor.shape)
            missing[0] = max_len - tensor.shape[0]

            if missing[0] > 0:
                padding_tensor = torch.full(missing, fill_value=pad_value, dtype=tensor.dtype, device=tensor.device)
                tensor = torch_cls.cat([tensor, padding_tensor], dim=0)

            new_batch.append(tensor)

        return torch_cls.stack(new_batch, dim=0)

    return batch

def zero_pad_collator(batch) -> Union[Dict[str, torch.Tensor], Tuple[torch.Tensor]]:
    datum = batch[0]

    # For strings
    if isinstance(datum, str):
        return batch

    # For tuples
    if isinstance(datum, tuple):
        return tuple(collate_tensors([b[i] for b in batch]) for i in range(len(datum)))

    # For dictionaries
    keys = datum.keys()
    return {k: collate_tensors([b[k] for b in batch]) for k in keys}

In [4]:
from pathlib import Path

from fontTools.ttLib import TTFont

class BaseTokenizer:

    def __init__(self,
                 tokens: List[str],
                 starting_index=None,
                 init_token="[CLS]",
                 eos_token="[SEP]",
                 pad_token="[PAD]",
                 unk_token="[UNK]"):
        if starting_index is None:
            starting_index = 4

        self.pad_token = pad_token
        self.bos_token = init_token
        self.eos_token = eos_token
        self.unk_token = unk_token

        self.i2s = {(i + starting_index): c for i, c in enumerate(tokens)}
        # Following the same ID scheme as JoeyNMT
        self.i2s[0] = self.unk_token
        self.i2s[1] = self.pad_token
        self.i2s[2] = self.bos_token
        self.i2s[3] = self.eos_token
        self.s2i = {c: i for i, c in self.i2s.items()}

        self.pad_token_id = self.s2i[self.pad_token]
        self.bos_token_id = self.s2i[self.bos_token]
        self.eos_token_id = self.s2i[self.eos_token]
        self.unk_token_id = self.s2i[self.unk_token]

    def __len__(self):
        return len(self.i2s)

    def vocab(self):
        return list(self.i2s.values())

    def text_to_tokens(self, text: str) -> List[str]:
        raise NotImplementedError()

    def tokens_to_text(self, tokens: List[str]) -> str:
        raise NotImplementedError()

    def tokenize(self, text: str, bos=False, eos=False):
        tokens = [self.s2i[c] for c in self.text_to_tokens(text)]
        if bos:
            tokens.insert(0, self.bos_token_id)
        if eos:
            tokens.append(self.eos_token_id)

        return tokens

    def detokenize(self, tokens: List[int]):
        if len(tokens) == 0:
            return ""
        if tokens[0] == self.bos_token_id:
            tokens = tokens[1:]
        if tokens[-1] == self.eos_token_id:
            tokens = tokens[:-1]

        try:
            padding_index = tokens.index(self.pad_token_id)
            tokens = tokens[:padding_index]
        except ValueError:
            pass

        return self.tokens_to_text([self.i2s[t] for t in tokens])

    def __call__(self, texts, is_tokenized=False, device=None):
        if not is_tokenized:
            all_tokens = [self.tokenize(text) for text in texts]
        else:
            all_tokens = texts.tolist()

        tokens_batch = zero_pad_collator([{
            "tokens_ids": torch.tensor(tokens, dtype=torch.long, device=device),
            "attention_mask": torch.ones(len(tokens), dtype=torch.bool, device=device),
            "positions": torch.arange(0, len(tokens), dtype=torch.int, device=device)
        } for tokens in all_tokens])
        # In transformers, 1 is mask, not 0
        tokens_batch["attention_mask"] = torch.logical_not(tokens_batch["attention_mask"])

        return tokens_batch

In [5]:
class HamNoSysTokenizer(BaseTokenizer):

    def __init__(self, starting_index=None, **kwargs):
        self.font_path = "./shared/tokenizers/hamnosys/HamNoSysUnicode.ttf"

        with TTFont(self.font_path) as font:
            tokens = [chr(key) for key in font["cmap"].getBestCmap().keys()]

        super().__init__(tokens=tokens, starting_index=starting_index, **kwargs)

    def text_to_tokens(self, text: str) -> List[str]:
        return list(text)

    def tokens_to_text(self, tokens: List[str]) -> str:
        return "".join(tokens)

In [6]:
# from text_to_pose.data import get_dataset
# from text_to_pose.model import IterativeTextGuidedPoseGenerationModel

## Data Preparation

In [7]:
from pose_format import Pose
from torch.utils.data import Dataset
from shared.tfds_dataset import ProcessedPoseDatum, get_tfds_dataset

In [8]:
class TextPoseDatum(TypedDict):
    id: str
    text: str
    pose: Pose
    length: int


class TextPoseDataset(Dataset):

    def __init__(self, data: List[TextPoseDatum]):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        datum = self.data[index]
        pose = datum["pose"]

        torch_body = pose.body.torch()
        pose_length = len(torch_body.data)

        return {
            "id": datum["id"],
            "text": datum["text"],
            "pose": {
                "obj": pose,
                "data": torch_body.data.tensor[:, 0, :, :],
                "confidence": torch_body.confidence[:, 0, :],
                "length": torch.tensor([pose_length], dtype=torch.float),
                "inverse_mask": torch.ones(pose_length, dtype=torch.int8)
            }
        }


def process_datum(datum: ProcessedPoseDatum) -> TextPoseDatum:
    text = datum["tf_datum"]["hamnosys"].numpy().decode('utf-8').strip()
    pose: Pose = datum["pose"]

    # Prune all leading frames containing only zeros
    for i in range(len(pose.body.data)):
        if pose.body.confidence[i].sum() != 0:
            if i != 0:
                pose.body.data = pose.body.data[i:]
                pose.body.confidence = pose.body.confidence[i:]
            break

    return {"id": datum["id"], "text": text, "pose": pose, "length": max(len(pose.body.data), len(text))}


In [9]:
def get_dataset(name="dicta_sign",
                poses="holistic",
                fps=25,
                split="train",
                components: List[str] = None,
                data_dir=None,
                max_seq_size=1000):
    data = get_tfds_dataset(name=name, poses=poses, fps=fps, split=split, components=components, data_dir=data_dir)

    data = [process_datum(d) for d in data]
    data = [d for d in data if d["length"] < max_seq_size]

    return TextPoseDataset(data)

In [10]:
from typing import List

import pytorch_lightning as pl
import torch
import torch.nn.functional as F
from torch import nn

In [11]:
from shared.models.pose_encoder import PoseEncoderModel

In [12]:
def masked_loss(loss_type, pose: torch.Tensor, pose_hat: torch.Tensor, confidence: torch.Tensor):
    # Loss by confidence. If missing joint, no loss. If less likely joint, less gradients.
    if loss_type == 'l1':
        error = torch.abs(pose - pose_hat).sum(-1)
    elif loss_type == 'l2':
        error = torch.pow(pose - pose_hat, 2).sum(-1)
    else:
        raise NotImplementedError()
    return (error * confidence).mean()


class DistributionPredictionModel(nn.Module):

    def __init__(self, input_size: int):
        super().__init__()

        self.fc_mu = nn.Linear(input_size, 1)
        self.fc_var = nn.Linear(input_size, 1)

    def forward(self, x: torch.Tensor):
        mu = self.fc_mu(x)
        if not self.training:  # In test time, just predict the mean
            return mu

        log_var = self.fc_var(x)
        # sample z from q
        std = torch.exp(log_var / 2)
        q = torch.distributions.Normal(mu, std)
        return q.rsample()


class IterativeTextGuidedPoseGenerationModel(pl.LightningModule):

    def __init__(self,
                 tokenizer,
                 pose_dims: (int, int) = (137, 2),
                 hidden_dim: int = 128,
                 text_encoder_depth=2,
                 pose_encoder_depth=4,
                 encoder_heads=2,
                 encoder_dim_feedforward=2048,
                 max_seq_size: int = 1000,
                 loss_type='l1'):
        super().__init__()

        self.tokenizer = tokenizer
        self.max_seq_size = max_seq_size

        # Embedding layers
        self.positional_embeddings = nn.Embedding(num_embeddings=max_seq_size, embedding_dim=hidden_dim)

        self.embedding = nn.Embedding(
            num_embeddings=len(tokenizer),
            embedding_dim=hidden_dim,
            padding_idx=tokenizer.pad_token_id,
        )

        self.pose_encoder = PoseEncoderModel(pose_dims=pose_dims,
                                             hidden_dim=hidden_dim,
                                             encoder_depth=pose_encoder_depth,
                                             encoder_heads=encoder_heads,
                                             encoder_dim_feedforward=encoder_dim_feedforward,
                                             max_seq_size=max_seq_size,
                                             dropout=0)

        # Encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim,
                                                   nhead=encoder_heads,
                                                   dim_feedforward=encoder_dim_feedforward,
                                                   batch_first=True)
        self.text_encoder = nn.TransformerEncoder(encoder_layer, num_layers=text_encoder_depth)

        # Predict sequence length
        self.seq_length = DistributionPredictionModel(hidden_dim)

        # Predict pose difference
        self.pose_diff_projection = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, self.pose_encoder.pose_dim),
        )

        # Loss
        self.loss_type = loss_type

    def encode_text(self, texts: List[str]):
        tokenized = self.tokenizer(texts, device=self.device)
        positional_embedding = self.positional_embeddings(tokenized["positions"])
        embedding = self.embedding(tokenized["tokens_ids"]) + positional_embedding
        encoded = self.text_encoder(embedding, src_key_padding_mask=tokenized["attention_mask"])
        seq_length = self.seq_length(torch.mean(encoded, dim=1))
        return {"data": encoded, "mask": tokenized["attention_mask"]}, seq_length

    def refine_pose_sequence(self, pose_sequence, text_encoding):
        batch_size, seq_length, _, _ = pose_sequence["data"].shape
        pose_encoding = self.pose_encoder(pose=pose_sequence, additional_sequence=text_encoding)
        pose_encoding = pose_encoding[:, :seq_length, :]

        # Predict desired change
        flat_pose_projection = self.pose_diff_projection(pose_encoding)
        return flat_pose_projection.reshape(batch_size, seq_length, *self.pose_encoder.pose_dims)

    def forward(self, text: str, first_pose: torch.Tensor, step_size: float = 0.5):
        text_encoding, sequence_length = self.encode_text([text])
        sequence_length = round(float(sequence_length))

        pose_sequence = {
            "data": first_pose.expand(1, sequence_length, *self.pose_encoder.pose_dims),
            "mask": torch.zeros([1, sequence_length], dtype=torch.bool),
        }
        while True:
            yield pose_sequence["data"][0]

            step = self.refine_pose_sequence(pose_sequence, text_encoding)
            pose_sequence["data"] = pose_sequence["data"] + step_size * step

    def training_step(self, batch, *unused_args, steps=100):
        return self.step(batch, *unused_args, steps=steps, name="train")

    def validation_step(self, batch, *unused_args, steps=100):
        return self.step(batch, *unused_args, steps=steps, name="validation")

    def step(self, batch, *unused_args, steps: int, name: str):
        text_encoding, sequence_length = self.encode_text(batch["text"])
        pose = batch["pose"]

        # Calculate sequence length loss
        sequence_length_loss = F.mse_loss(sequence_length, pose["length"]) / 10000

        # Repeat the first frame for initial prediction
        batch_size, pose_seq_length, _, _ = pose["data"].shape
        pose_sequence = {
            "data": torch.stack([pose["data"][:, 0]] * pose_seq_length, dim=1),
            "mask": torch.logical_not(pose["inverse_mask"])
        }

        refinement_loss = 0
        for _ in range(steps):
            pose_sequence["data"] = pose_sequence["data"].detach()  # Detach from graph
            l1_gold = pose["data"] - pose_sequence["data"]
            l1_predicted = self.refine_pose_sequence(pose_sequence, text_encoding)
            refinement_loss += masked_loss(self.loss_type, l1_gold, l1_predicted, confidence=pose["confidence"])

            step_size = 1 / steps
            l1_step = l1_gold if name == "validation" else l1_predicted
            pose_sequence["data"] = pose_sequence["data"] + step_size * l1_step

            if name == "train":  # add just a little noise while training
                pose_sequence["data"] = pose_sequence["data"] + torch.randn_like(pose_sequence["data"]) * 1e-4

        self.log(name + "_seq_length_loss", sequence_length_loss, batch_size=batch_size)
        self.log(name + "_refinement_loss", refinement_loss, batch_size=batch_size)
        loss = refinement_loss + sequence_length_loss
        self.log(name + "_loss", loss, batch_size=batch_size)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

In [13]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from torch.utils.data import DataLoader

In [14]:
LOGGER = WandbLogger(project="text-to-pose", log_model=False, offline=False)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ········


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Nguyen Thanh Loc/.netrc


In [15]:
train_dataset = get_dataset(split="train[10:]")
train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, collate_fn=zero_pad_collator)

INFO:absl:Generating dataset dicta_sign (C:\Users\Nguyen Thanh Loc\tensorflow_datasets\dicta_sign\holistic-25\1.0.0)
c:\users\nguyen thanh loc\appdata\local\programs\python\python39\lib\site-packages\sign_language_datasets\datasets\warning.py:5: UserWarning: This library provides access to data sets without claiming ownership over them or defining their licensing terms. Users who download data are responsible for checking the license of each individual data set.
  warnings.warn(


Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]INFO:absl:Downloading https://www.sign-lang.uni-hamburg.de/dicta-sign/portal/concepts/concepts_eng.html into C:\Users\Nguyen Thanh Loc\tensorflow_datasets\downloads\sign-lang.uni-hamb.de_dict-sign_port_concgRUP1VnkJAI21E4dnk2KQC_oyHt4pQbuIXI6_q22L4Y.html.tmp.a3d54aa168c74e45a43eb89b63b81f2d...
Dl Completed...: 100%|█████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.71s/ url]
Dl Size...: 0 MiB [00:02, ? MiB/s]
Dl Completed...: 100%|█████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.78s/ url]
Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]INFO:absl:Downloading https://nlp.biu.ac.il/~amit/datasets/poses/holistic/dicta_sign.tar.gz into C:\Users\Nguyen Thanh Loc\tensorflow_datasets\downloads\nlp.biu.ac.il_amit_data_pose_holi_dict_L2t9fho0Yyr37pUDesJLkvcgXAwidfgqE3Qzsf3

Dl Size...:   1%|▋                                                               | 22/2230 [01:46<3:06:32,  5.07s/ MiB]

Extraction completed...: 0 file [01:46, ? file/s]
Dl Size...:   1%|▋                                                               | 23/2230 [01:49<2:44:14,  4.47s/ MiB]

Extraction completed...: 0 file [01:49, ? file/s]
Dl Size...:   1%|▋                                                               | 24/2230 [01:53<2:37:12,  4.28s/ MiB]

Extraction completed...: 0 file [01:53, ? file/s]
Dl Size...:   1%|▋                                                               | 25/2230 [02:00<3:09:28,  5.16s/ MiB]

Extraction completed...: 0 file [02:00, ? file/s]
Dl Size...:   1%|▋                                                               | 26/2230 [02:04<3:02:03,  4.96s/ MiB]

Extraction completed...: 0 file [02:04, ? file/s]
Dl Size...:   1%|▊                                                               | 27/2230 [02:08<2:42:59,  4.44s/ MiB]

Extraction completed...:

Dl Size...:   2%|█▍                                                              | 49/2230 [04:30<5:25:16,  8.95s/ MiB]

Extraction completed...: 0 file [04:30, ? file/s]
Dl Size...:   2%|█▍                                                              | 50/2230 [04:39<5:23:47,  8.91s/ MiB]

Extraction completed...: 0 file [04:39, ? file/s]
Dl Size...:   2%|█▍                                                              | 51/2230 [04:44<4:35:33,  7.59s/ MiB]

Extraction completed...: 0 file [04:44, ? file/s]
Dl Size...:   2%|█▍                                                              | 52/2230 [04:47<3:48:31,  6.30s/ MiB]

Extraction completed...: 0 file [04:47, ? file/s]
Dl Size...:   2%|█▌                                                              | 53/2230 [04:51<3:23:43,  5.61s/ MiB]

Extraction completed...: 0 file [04:51, ? file/s]
Dl Size...:   2%|█▌                                                              | 54/2230 [04:56<3:15:04,  5.38s/ MiB]

Extraction completed...:

Dl Size...:   3%|██▏                                                             | 76/2230 [06:39<2:00:32,  3.36s/ MiB]

Extraction completed...: 0 file [06:39, ? file/s]
Dl Size...:   3%|██▏                                                             | 77/2230 [06:41<1:54:17,  3.19s/ MiB]

Extraction completed...: 0 file [06:41, ? file/s]
Dl Size...:   3%|██▏                                                             | 78/2230 [06:44<1:50:06,  3.07s/ MiB]

Extraction completed...: 0 file [06:44, ? file/s]
Dl Size...:   4%|██▎                                                             | 79/2230 [06:47<1:46:45,  2.98s/ MiB]

Extraction completed...: 0 file [06:47, ? file/s]
Dl Size...:   4%|██▎                                                             | 80/2230 [06:50<1:46:45,  2.98s/ MiB]

Extraction completed...: 0 file [06:50, ? file/s]
Dl Size...:   4%|██▎                                                             | 81/2230 [06:53<1:46:38,  2.98s/ MiB]

Extraction completed...:

Dl Size...:   5%|██▉                                                            | 103/2230 [08:32<2:45:41,  4.67s/ MiB]

Extraction completed...: 0 file [08:32, ? file/s]
Dl Size...:   5%|██▉                                                            | 104/2230 [08:39<3:08:33,  5.32s/ MiB]

Extraction completed...: 0 file [08:39, ? file/s]
Dl Size...:   5%|██▉                                                            | 105/2230 [08:50<4:09:00,  7.03s/ MiB]

Extraction completed...: 0 file [08:50, ? file/s]
Dl Size...:   5%|██▉                                                            | 106/2230 [08:59<4:31:55,  7.68s/ MiB]

Extraction completed...: 0 file [08:59, ? file/s]
Dl Size...:   5%|███                                                            | 107/2230 [09:06<4:19:17,  7.33s/ MiB]

Extraction completed...: 0 file [09:06, ? file/s]
Dl Size...:   5%|███                                                            | 108/2230 [09:10<3:47:35,  6.44s/ MiB]

Extraction completed...:

Dl Size...:   6%|███▋                                                           | 130/2230 [10:32<2:17:12,  3.92s/ MiB]

Extraction completed...: 0 file [10:32, ? file/s]
Dl Size...:   6%|███▋                                                           | 131/2230 [10:36<2:14:10,  3.84s/ MiB]

Extraction completed...: 0 file [10:36, ? file/s]
Dl Size...:   6%|███▋                                                           | 132/2230 [10:40<2:13:45,  3.83s/ MiB]

Extraction completed...: 0 file [10:40, ? file/s]
Dl Size...:   6%|███▊                                                           | 133/2230 [10:43<2:15:17,  3.87s/ MiB]

Extraction completed...: 0 file [10:43, ? file/s]
Dl Size...:   6%|███▊                                                           | 134/2230 [10:50<2:41:57,  4.64s/ MiB]

Extraction completed...: 0 file [10:50, ? file/s]
Dl Size...:   6%|███▊                                                           | 135/2230 [10:54<2:35:47,  4.46s/ MiB]

Extraction completed...:

Dl Size...:   7%|████▍                                                          | 157/2230 [12:47<2:37:16,  4.55s/ MiB]

Extraction completed...: 0 file [12:47, ? file/s]
Dl Size...:   7%|████▍                                                          | 158/2230 [12:52<2:40:03,  4.63s/ MiB]

Extraction completed...: 0 file [12:52, ? file/s]
Dl Size...:   7%|████▍                                                          | 159/2230 [12:59<3:01:29,  5.26s/ MiB]

Extraction completed...: 0 file [12:59, ? file/s]
Dl Size...:   7%|████▌                                                          | 160/2230 [13:04<2:56:40,  5.12s/ MiB]

Extraction completed...: 0 file [13:04, ? file/s]
Dl Size...:   7%|████▌                                                          | 161/2230 [13:11<3:14:20,  5.64s/ MiB]

Extraction completed...: 0 file [13:11, ? file/s]
Dl Size...:   7%|████▌                                                          | 162/2230 [13:18<3:32:38,  6.17s/ MiB]

Extraction completed...:

Dl Size...:   8%|█████▏                                                         | 184/2230 [15:57<5:46:30, 10.16s/ MiB]

Extraction completed...: 0 file [15:57, ? file/s]
Dl Size...:   8%|█████▏                                                         | 185/2230 [16:08<5:50:27, 10.28s/ MiB]

Extraction completed...: 0 file [16:08, ? file/s]
Dl Size...:   8%|█████▎                                                         | 186/2230 [16:21<6:19:27, 11.14s/ MiB]

Extraction completed...: 0 file [16:21, ? file/s]
Dl Size...:   8%|█████▎                                                         | 187/2230 [16:32<6:14:44, 11.01s/ MiB]

Extraction completed...: 0 file [16:32, ? file/s]
Dl Size...:   8%|█████▎                                                         | 188/2230 [16:38<5:29:08,  9.67s/ MiB]

Extraction completed...: 0 file [16:38, ? file/s]
Dl Size...:   8%|█████▎                                                         | 189/2230 [16:45<4:57:21,  8.74s/ MiB]

Extraction completed...:

ChunkedEncodingError: ("Connection broken: ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)", ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))